In [0]:
from pyspark.sql import SparkSession
from pyspark.dbutils import DBUtils
from datetime import datetime
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("DSS-PROJECT").getOrCreate()

In [0]:
#creating mount point to access data in adls

dbutils.fs.mount(
  source=f"wasbs://project-data@pharmacydata2024.blob.core.windows.net/",
  mount_point="/mnt/dss-project-works",
  extra_configs={"fs.azure.account.key.pharmacydata2024.blob.core.windows.net": "DECBZEPbRnh8pmQGIHIRyH9QpredTuDf6nlHoQOn8pHqb1YBPzBHDrvo1uKIojLYh226lFaNS62B+AStdZxCCw=="}
)

In [0]:
mount_point="/mnt/dss-project-works"
dbutils.fs.ls(mount_point)

[FileInfo(path='dbfs:/mnt/dss-project-works/project-data/', name='project-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/dss-project-works/raw-data/', name='raw-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/dss-project-works/test_output_from_adb/', name='test_output_from_adb/', size=0, modificationTime=0)]

In [0]:
dbutils.fs.ls("dbfs:/mnt/dss-project-works/raw-data/") 
source="/mnt/dss-project-works/raw-data/"

In [0]:
customerdetails_file="/mnt/dss-project-works/raw-data/CustomerDetails.csv"
df_customerdetails=spark.read.format("csv").option("header","true").option("inferSchema","true").load(customerdetails_file)
df_customerdetails.show()

+----------+--------------------+-----------+--------------+--------------------+-------------+-----------+--------------------+----------+
|CustomerID|                Name|CustomerAge|CustomerGender|         FullAddress|         City|PhoneNumber|             EmailID|       DOB|
+----------+--------------------+-----------+--------------+--------------------+-------------+-----------+--------------------+----------+
|      U001|      Teresa Coleman|         90|        Female|589 Julie Cape, T...| Philadelphia| 6805671430| stacy06@example.com|1961-05-16|
|      U002|      April Thompson|         27|          Male|819 Davila Garden...|New York City| 9035430519|   wsoto@example.org|1963-12-25|
|      U003|      Brittany Patel|         55|          Male|219 Powell Throug...|      Chicago| 5533860370|oliverelizabeth@e...|1934-07-19|
|      U004|   Ricky Christensen|         77|          Male|745 Courtney Clif...|  San Antonio| 3968078573|rbarrera@example.com|1944-10-17|
|      U005|        

In [0]:
inventory_file="/mnt/dss-project-works/raw-data/Inventory_Data_2024.csv"
df_inventory=spark.read.format("csv").option("header", "true").option("inferSchema","true").load(inventory_file)
df_inventory.show()

+-------+-------------+---------+----------------+--------------------+------------+------------+---------------+---------------+---------------+
|StoreID|StoreLocation|ProductID|     ProductName|     ProductCategory|CurrentStock|ReorderLevel|ReorderQuantity|LastRestockDate|ProjectedDemand|
+-------+-------------+---------+----------------+--------------------+------------+------------+---------------+---------------+---------------+
|   S001| Philadelphia|     P001|    Clotrimazole|Dermatological (O...|          29|           7|             25|     2024-02-04|             32|
|   S001|    San Diego|     P001|      Ranitidine|Gastrointestinal ...|          93|           9|             24|     2024-01-01|             25|
|   S001|      Houston|     P001|       Ibuprofen|Pain Relief (OTC ...|          15|           9|             20|     2024-07-18|             44|
|   S001|New York City|     P001|      Metoprolol|Cardiovascular Di...|          51|           5|             24|     2024-0

In [0]:
pos_file="/mnt/dss-project-works/raw-data/POS_Sales_Data_2024.csv"
df_pos=spark.read.format("csv").option("header","true").option("inferSchema","true").load(pos_file)
df_pos.show()

+-------------+-------+-------------+---------+----------+-----------+--------------+---------+--------------------+-------------+--------+---------+----------------+-----------+---------------+---------+-------------------+
|TransactionID|StoreID|StoreLocation|CashierID|CustomerID|CustomerAge|CustomerGender|ProductID|     ProductCategory|  ProductName|Quantity|UnitPrice|       Timestamp|TotalAmount|DiscountApplied|TaxAmount|TotalAmountAfterTax|
+-------------+-------+-------------+---------+----------+-----------+--------------+---------+--------------------+-------------+--------+---------+----------------+-----------+---------------+---------+-------------------+
|        T0001|   S010|  Los Angeles|     C005|      U051|         62|          Male|     P005|Cancer Treatment ...|    Metformin|       4|   115.25|24-06-2024 11:02|      461.0|          25.84|    32.27|             467.43|
|        T0002|   S008| Philadelphia|     C009|      U018|         29|        Female|     P036|Cance

In [0]:
vendor_file="/mnt/dss-project-works/raw-data/Product_Categorywise_Vendor_Data_2024.csv"
df_vendor=spark.read.format("csv").option("header","true").option("inferSchema","true").load(vendor_file)
df_vendor.show()

+--------+--------------------+--------------------+-----------+
|VendorID|          VendorName|     ProductCategory| SupplyRate|
+--------+--------------------+--------------------+-----------+
|    V001|        Ellis-Farmer|Cancer Treatment ...|1.038399201|
|    V002|         Edwards PLC|Eye Health (OTC &...|1.097562743|
|    V003|Smith, Watkins an...|Cancer Treatment ...|0.960920011|
|    V004|           Baird Ltd|Mental Health (Pr...|0.818460498|
|    V005|            Yang Inc|Pain Relief (OTC ...| 1.19648295|
|    V006|       Sandoval-Wang|Antibiotics (Pres...|1.196316238|
|    V007|         Munoz-Ramos|Cardiovascular Di...|1.105356666|
|    V008|   Hamilton-Callahan|Diabetes (Prescri...|0.873965169|
|    V009|White, Burke and ...|AIDS Treatment (P...| 0.83970798|
|    V010|       Stewart Group|Diabetes (Prescri...|0.817664363|
+--------+--------------------+--------------------+-----------+



In [0]:
df_vendors_filtered=df_vendor.filter(col("SupplyRate")>=1.0)
df_vendors_filtered.show()

+--------+-------------+--------------------+-----------+
|VendorID|   VendorName|     ProductCategory| SupplyRate|
+--------+-------------+--------------------+-----------+
|    V001| Ellis-Farmer|Cancer Treatment ...|1.038399201|
|    V002|  Edwards PLC|Eye Health (OTC &...|1.097562743|
|    V005|     Yang Inc|Pain Relief (OTC ...| 1.19648295|
|    V006|Sandoval-Wang|Antibiotics (Pres...|1.196316238|
|    V007|  Munoz-Ramos|Cardiovascular Di...|1.105356666|
+--------+-------------+--------------------+-----------+



In [0]:
mount_point="/mnt/dss-project-works"
dbutils.fs.ls(mount_point)

[FileInfo(path='dbfs:/mnt/dss-project-works/project-data/', name='project-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/dss-project-works/raw-data/', name='raw-data/', size=0, modificationTime=0),
 FileInfo(path='dbfs:/mnt/dss-project-works/test_output_from_adb/', name='test_output_from_adb/', size=0, modificationTime=0)]

In [0]:
#repartition to only 1 part file in the output

test="/mnt/dss-project-works/test_output_from_adb/"
df_vendors_filtered.repartition(1).write.mode("overwrite").format('csv').save(test)

In [0]:
#creating a database to create a delta table inside it
%sql
CREATE DATABASE IF NOT EXISTS test

In [0]:
%sql
SELECT * FROM test.df_vendors_filtered

VendorID,VendorName,ProductCategory,SupplyRate
V001,Ellis-Farmer,Cancer Treatment (Prescription),1.038399201
V002,Edwards PLC,Eye Health (OTC & Prescription),1.097562743
V005,Yang Inc,Pain Relief (OTC & Prescription),1.19648295
V006,Sandoval-Wang,Antibiotics (Prescription),1.196316238
V007,Munoz-Ramos,Cardiovascular Diseases (Prescription),1.105356666


In [0]:
%sql
DESCRIBE project_adb.test.df_vendors_filtered;


col_name,data_type,comment
VendorID,string,null
VendorName,string,null
ProductCategory,string,null
SupplyRate,double,null


In [0]:
#Creating a delta table

%sql
CREATE TABLE IF NOT EXISTS test.df_vendors_filtered 
(
  VendorID VARCHAR(10),
  VendorName VARCHAR(100),
  ProductCategory VARCHAR(100),
  SupplyRate DECIMAL(8,6)
)
USING DELTA

In [0]:
#Saving delta frame to a delta table

df_vendors_filtered.write.mode("overwrite") \
    .option("overwriteSchema", "true") \
    .format('delta') \
    .saveAsTable("test.df_vendors_filtered")

In [0]:
df_vendors_filtered.printSchema()

root
 |-- VendorID: string (nullable = true)
 |-- VendorName: string (nullable = true)
 |-- ProductCategory: string (nullable = true)
 |-- SupplyRate: double (nullable = true)



In [0]:
%sql
SELECT * FROM test.df_vendors_filtered

VendorID,VendorName,ProductCategory,SupplyRate
V001,Ellis-Farmer,Cancer Treatment (Prescription),1.038399201
V002,Edwards PLC,Eye Health (OTC & Prescription),1.097562743
V005,Yang Inc,Pain Relief (OTC & Prescription),1.19648295
V006,Sandoval-Wang,Antibiotics (Prescription),1.196316238
V007,Munoz-Ramos,Cardiovascular Diseases (Prescription),1.105356666


In [0]:
%sql
DESCRIBE EXTENDED test.df_vendors_filtered

col_name,data_type,comment
VendorID,string,null
VendorName,string,null
ProductCategory,string,null
SupplyRate,double,null
,,
# Delta Statistics Columns,,
Column Names,"VendorID, VendorName, ProductCategory, SupplyRate",
Column Selection Method,first-32,
,,
# Detailed Table Information,,


In [0]:
#Loading existing delta table contents to a dataframe

df=spark.table("project_adb.test.df_vendors_filtered")
display(df)

VendorID,VendorName,ProductCategory,SupplyRate
V001,Ellis-Farmer,Cancer Treatment (Prescription),1.038399201
V002,Edwards PLC,Eye Health (OTC & Prescription),1.097562743
V005,Yang Inc,Pain Relief (OTC & Prescription),1.19648295
V006,Sandoval-Wang,Antibiotics (Prescription),1.196316238
V007,Munoz-Ramos,Cardiovascular Diseases (Prescription),1.105356666


In [0]:
#Save output file in different file formats with name like 'part-00000' in adls

df2=spark.table("project_adb.test.df_vendors_filtered")
df2.write.mode("overwrite").format("csv").save("/mnt/dss-project-works/project-data/test_output_from_adb/delta-table-test-output/df_vendors_filtered.csv",header=True)

In [0]:
#Custom Name File Renaming in Databricks with date format directory

from pyspark.sql.functions import current_date

files = dbutils.fs.ls("dbfs:/mnt/dss-project-works/project-data/test_output_from_adb/delta-table-test-output/")

current_date_value = spark.sql("SELECT date_format(current_date(), 'yyyy-MM-dd')").collect()[0][0]
print(current_date_value)

dbutils.fs.mkdirs(f"dbfs:/mnt/dss-project-works/project-data/test_output_from_adb/{current_date_value}")

for file in files:
    if "part-00000-tid-" in file.name:
        dbutils.fs.cp(file.path,destination_path)
        break


2024-09-24


In [0]:
#Save output file in different file formats

df=spark.table("project_adb.test.df_vendors_filtered")
df.write.mode("overwrite").format("parquet").save("/mnt/dss-project-works/project-data/test_output_from_adb/delta-table-test-output/")

df1=spark.table("project_adb.test.df_vendors_filtered")
df1.write.mode("overwrite").format("csv").save("/mnt/dss-project-works/project-data/test_output_from_adb/delta-table-test-output/")